# Linear Elasticity Deterministic Inverse Problem

This notebook uses `hippylib` to set up the parametric PDE map and eventually solve a deterministic inverse problem for an uncertain elastic modulus in a linear elasticity problem


## Linear elasticity

We can derive linear elasticity as the minimization of strain energy

$$
\min_u \int_\Omega \psi(\varepsilon(u)) dx - \int_\Omega f\cdot u dx - \int_{\Gamma_N} t \cdot n ds(x)
$$
where $\varepsilon(u) = \frac{1}{2}(\nabla u + \nabla u^T)$
where $u$ is restricted to an appropriate space to satisfy given Dirichlet boundary conditions, $f$ is a body load, and $t$ is a prescribed traction condition. Additionally:

$$
\psi(\boldsymbol{\varepsilon}) = \frac{\lambda}{2} \bigl(\mathrm{tr}(\boldsymbol{\varepsilon})\bigr)^2 + \mu \,\mathrm{tr}(\boldsymbol{\varepsilon}^2)
$$

The minimizer gives us the following system

$$ -\nabla \cdot \sigma(u) = f$$
$$ u = u_D \text{ on } \Gamma_D$$
$$ \sigma \cdot n = t \text{ on } \Gamma_N$$

$$ \sigma(u) = \lambda (\nabla \cdot u) I + 2\mu \varepsilon(u)$$

## Parametric PDE Mapping

In our case we parametrize our model by the elastic modulus:

$$ E = \frac{\mu(3\lambda + 2\mu)}{\lambda + \mu} $$

and we assume the Poisson's ration $\nu = \frac{\lambda}{2(\lambda + \mu)} = 0.4$ to be fixed.

We generate training data samples of $E$ using Gaussian random fields $m$. We assume $E = 1 + \exp(m)$, in order to maintain the coercivity (well-posedness) of the PDE.

## Operator Learning Primary Goal

**We are interested in the mapping from $m$ to the associated displacement field $u(m)$ over a given distribution of $m$**

$$ m \mapsto u(m) \quad \text{ over } \quad m \sim \mu$$


## Finite Element Method

For all tutorials we will utilize the finite element method through the open source library `fenics`


In [ ]:
# MIT License
# Copyright (c) 2025
#
# This is part of the dino_tutorial package
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND.
# For additional questions contact Thomas O'Leary-Roseberry

import dolfin as dl
import ufl
import math
import numpy as np
import matplotlib.pyplot as plt
import sys, os
import time
sys.path.append( os.environ.get('HIPPYLIB_PATH', "../") )
import hippylib as hp

from linear_elasticity_model import *

# sys.path.append( os.environ.get('HIPPYFLOW_PATH'))
# import hippyflow as hf

import logging
logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
dl.set_log_active(False)

np.random.seed(seed=42)


### Key Objects

- `settings` defines auxiliary hyperparameters for the PDE problem
- `hippylib.model` is an object that contains all of the sub-objects that define the PDE problem and the associated inverse problem. Within the model we have
    - `hippylib.model.prior` defines the reference distribution for the parameter. We will utilize Gaussian priors, as they allow for the definition of infinite-dimensionally consistent measures for spatially correlated random variables.
        $$ m\sim \mu = \mathcal{N}(\overline{m},\mathcal{C})$$ 
    - `hippylib.model.problem` defines the PDE problem variationally using the finite element method. This contains all of the infrastructure for solving the PDE, including the solver (linear and nonlinear) as well as additional methods that will be utilized to compute derivatives later on.
      $$ \text{Given } m \sim \mu, \quad \text{evaluate} \quad m\mapsto u(m)$$


### Note on extensibility

All of the code is written to intentionally abstract away the PDE model and reference distribution. This is to make it easier to rapid-prototype other PDE problems later with a BYO-PDE problem.
    

## Load the model problem 

In [ ]:
# Define some basic settings for 
settings = linear_elasticity_settings()
# Define model
model = linear_elasticity_model(settings)

# This is a python list of three function spaces
# [Vh_STATE, Vh_PARAMETER, Vh_ADJOINT]
Vh = model.problem.Vh


## The prior / reference distribution $\mu$

We seek to model the elastic modulus field as a nonlinear function of a Gaussian random field.
    $$ E(m) = 1 + \exp(m) $$

### Gaussian random fields

We model $m$ as a spatially correlated random field. Towards this end we utilize a self-adjoint fractional elliptic PDE operator for the covariance to impart local spatial coupling.

$$ m \sim \mathcal{N}(\overline{m},\mathcal{C}) $$

$$ \text{Mean function:} \quad \overline{m} $$

$$ \text{Covariance:} \quad \mathcal{C} = (I - \Delta)^{-\alpha} \quad \text{+ B.C.s}$$ 

Sampling from Gaussians: Given white noise $\xi$, then 
$$ m = \overline{m} + \mathcal{C}^\frac{1}{2} \xi \sim \mathcal{N}(\overline{m},\mathcal{C})$$

So we need to be able to apply $\mathcal{C}^\frac{1}{2}$. Choosing $\alpha > \frac{d}{2}$ where $\Omega \subset \mathbb{R}^d$ leads to **discretization-consistency**. The choice of $\alpha = 2$ is suitable for discretization consistency for both $\mathbb{R}^2$ and $\mathbb{R}^3$ and is utilized commonly. Thus the application of $\mathcal{C}^\frac{1}{2}$ corresponds to a PDE solve:

$$ (I - \Delta)^{-1}.$$

This construction costs for this elliptic PDE solution operator can be amortized over its application to many right-hand sides. This allows for substantial savings when using sparse solvers or preconditioners. 

In [ ]:
# Generate data:
############
noise = dl.Vector()
model.prior.init_vector(noise,"noise")
hp.parRandom.normal(1., noise)
mtrue = dl.Vector()
model.prior.init_vector(mtrue, 0)
model.prior.sample(noise, mtrue)

# # plot mtrue, mean 
# objs = [dl.Function(model.problem.Vh[hp.PARAMETER],mtrue), 
#         dl.Function(model.problem.Vh[hp.PARAMETER],model.prior.mean)]
# mytitles = ["True Parameter", "Prior mean"]
# hp.utils.nb.multi1_plot(objs, mytitles)
# plt.show()

cbar = dl.plot(dl.Function(model.problem.Vh[hp.PARAMETER],model.prior.mean), title="Mean field")
# _ = plt.colorbar(cbar)
# plt.show()

cbar = dl.plot(dl.Function(model.problem.Vh[hp.PARAMETER],mtrue), title="A sample")
# _ = plt.colorbar(cbar)
plt.show()

## Solve the PDE and look at the associated displacement field

In [ ]:

utrue = model.problem.generate_state()
x = [utrue, mtrue, None]

model.problem.solveFwd(x[hp.STATE], x)

plt.figure(figsize=(15,11))
u_norm = get_unorm(model, utrue)

u_plot = dl.Function(Vh[hp.STATE])
u_plot.vector().zero()
u_plot.vector().axpy(1.,utrue)

vmax = u_norm.vector().max()
vmin = u_norm.vector().min()

# Project onto a scalar function space. Often, u0.function_space() is a scalar subspace.
plt.figure(figsize=(15,5))
plt.subplot(221)
cbar = dl.plot(u_plot, mode="displacement", title="Displacement")
_ = plt.colorbar(cbar)


# hp.utils.nb.plot(u_norm, mytitle="Absolute Value (u)", subplot_loc=221, vmin=vmin, vmax=vmax)
# norm = np.linalg.norm(model.misfit.d.get_local().reshape((-1, 2)), axis=1)
# tmp = dl.Vector(dl.MPI.comm_world, len(norm))
# tmp.set_local(norm)
# hp.utils.nb.plot_pts(model.targets, tmp, mytitle="Observations (u)", subplot_loc=222, vmin=vmin, vmax=vmax)


In [ ]:
dl.plot(u_plot, mode="displacement", title="Displacement")

## Inverse Problem

We have pointwise observations of the displacements $u \in \mathcal{U}$. The observation operator is $B:\mathcal{U} \rightarrow \mathbb{R}^{d}$. The observational data $\mathbf{d}$ are assumed to agree with the true observations up to some irreducible observational noise $\xi \sim \mathcal{N}(0,\Gamma)$. That is $\mathbf{d} = Bu(m_\text{true}) + \xi$

The minimization problem is 

$$ \min_m \frac{1}{2}\|Bu(m) - \mathbf{d}\|^2_{\Gamma^{-1}} + \frac{1}{2}\|m - \overline{m}\|^2_{\mathcal{C}^{-1}}$$

where $\mathcal{C}^{-1}$ is a weighting for a Tikhonov regularization, and in the Bayesian context we identify it with the covariance of a Gaussian prior, e.g., $\mu_\text{prior} = \mathcal{N}(\overline{m},\mathcal{C})$.

## Observation operator $B$ and observational data $\mathbf{d}$

In [ ]:
import hippylib.utils.nb as nb

data = model.misfit.B*x[hp.STATE]
# MAX = data.norm("linf")
# noise_std_dev = rel_noise * MAX
hp.parRandom.normal_perturb(settings['noise_variance']**0.5, data)
model.misfit.d = data

vmax = max( utrue.max(), model.misfit.d.max() )
vmin = min( utrue.min(), model.misfit.d.min() )

plt.figure(figsize=(15,5))
nb.plot(dl.Function(model.problem.Vh[hp.STATE], utrue), mytitle="True State", subplot_loc=121, vmin=vmin, vmax=vmax)
norm = np.linalg.norm(model.misfit.d.get_local().reshape((-1, 2)), axis=1)
tmp = dl.Vector(dl.MPI.comm_world, len(norm))
tmp.set_local(norm)
nb.plot_pts(model.targets, tmp, mytitle="Observed Data $\mathbf{d}$", subplot_loc=122, vmin=vmin, vmax=vmax)

    
plt.show()

## Solve the nonlinear least-squares problem using Newton

Define the objective function

$$ F(m) = \frac{1}{2}\|Bu(m) - \mathbf{d}\|^2_{\Gamma^{-1}} + \frac{1}{2}\|m - \overline{m}\|^2_{\mathcal{C}^{-1}}$$

We can then compute gradients $\nabla F(m)$, and Hessians $\nabla^2 F(m)$, and minimize $F(m)$ utilizing a Newton method

$$ m_{k+1} = m_k +\alpha_k p_k$$
$$ p_k = -\nabla^2 F(m_k)^{-1}\nabla F(m_k)$$

The derivatives are typically computed using **adjoint methods** since they require differentiating through an implicitly defined PDE solution operator. The costs of computing (implicit) derivatives are then measured in PDE solutions.

- each gradient requires an additional (linearized) adjoint PDE solution.
- each Hessian vector product requires two additional linearized PDE solutions (fwd incremental PDE and adjoint incremental PDE).

We can avoid explicitly inverting the Hessian by utilizing an inexact Newton conjugate gradients method. Additionally we can utilize a stopping criterion for the linearized solve via the so-called Eisenstat--Walker conditions

$$ \|\nabla^2 F(m_k)p_k + \nabla F(m_k)\| \leq c\|\nabla F(m_k)\| $$

so we require that the linear solves are asymptotically exact as $\|\nabla F(m_k)\| \rightarrow 0$, while allowing for computational savings far from the solution $\|\nabla F(m^\star)\| = 0$



In [ ]:
# Compute MAP point
m = model.prior.mean.copy()
solver = hp.ReducedSpaceNewtonCG(model)
solver.parameters["rel_tolerance"] = 1e-6
solver.parameters["abs_tolerance"] = 1e-12
solver.parameters["max_iter"]      = 100
solver.parameters["GN_iter"] = 20
solver.parameters["globalization"] = "LS"
solver.parameters["LS"]["c_armijo"] = 1e-4

    
x = solver.solve([None, m, None])
    
if solver.converged:
    print( "\nConverged in ", solver.it, " iterations.")
else:
    print( "\nNot Converged")

print( "Termination reason: ", solver.termination_reasons[solver.reason] )
print( "Final gradient norm: ", solver.final_grad_norm )
print( "Final cost: ", solver.final_cost )

In [ ]:
# Plot MAP point
plt.figure(figsize=(15,5))
nb.plot(dl.Function(model.problem.Vh[hp.PARAMETER],mtrue), subplot_loc=121,mytitle="True Parameter")
nb.plot(dl.Function(model.problem.Vh[hp.PARAMETER], x[hp.PARAMETER]), subplot_loc=122,mytitle="MAP Parameter")
plt.show()

## Extra credit! Let's look at the Hessian at the optimum $\nabla^2F(m^\star)$

In [ ]:
model.setPointForHessianEvaluations(x, gauss_newton_approx=False)
Hmisfit = hp.ReducedHessian(model, misfit_only=True)
k = 50
p = 20
print( "Single/Double Pass Algorithm. Requested eigenvectors: {0}; Oversampling {1}.".format(k,p) )

Omega = hp.MultiVector(x[hp.PARAMETER], k+p)
hp.parRandom.normal(1., Omega)
lmbda, V = hp.doublePassG(Hmisfit, model.prior.R, model.prior.Rsolver, Omega, k)

posterior = hp.GaussianLRPosterior(model.prior, lmbda, V)
posterior.mean = x[hp.PARAMETER]

plt.plot(range(0,k), lmbda, 'b*', range(0,k+1), np.ones(k+1), '-r')
plt.yscale('log')
plt.xlabel('number')
plt.ylabel('eigenvalue')

nb.plot_eigenvectors(model.problem.Vh[hp.PARAMETER], V, mytitle="Eigenvector", which=[0,1,2,5,10,15])